In [3]:
import datetime
from sqlalchemy import Column, Integer, String, Float, DateTime, create_engine
from sqlalchemy.ext.declarative import declarative_base

engine = create_engine('sqlite:///db/projektai.db')
Base = declarative_base()


# LENTELES SUKURIMAS

In [6]:
class Projektas(Base):
    __tablename__ = 'projektas'
    id = Column(Integer, primary_key=True)
    name = Column("pavadinimas", String)
    price = Column("kainas", Float)
    created_date = Column("sukurta", DateTime, default=datetime.datetime.utcnow)

    def __init__(self, name, price):
        self.name = name
        self.price = price

    def __repr__(self) -> str:
        return f"({self.id}, {self.name}, {self.created_date})"


In [7]:
Base.metadata.create_all(engine)

In [8]:
from sqlalchemy.orm import sessionmaker
# Session = sessionmaker(bind=engine)
# session = Session()
# Dar tą patį galima užrašyti ir taip:
session = sessionmaker(bind=engine)()

# CRUDE - create read update delete. 

Standartinis kiekvienos lenteles veiksmu paketas

In [9]:
# Create
projektas1 = Projektas("Pirmas projektas", 20000)
# į session-ą įdedame savo projektas1:
session.add(projektas1)
# session.commit atliks įterpimo procedūrą:
session.commit()


In [10]:
projektas2 = Projektas("Antras projektas", 55000)
session.add(projektas2)
session.commit()

In [11]:
# read one Projektų nuskaitymas iš duomenų bazės
projektas = session.query(Projektas).get(1)
projektas.price
#type(projektas.price)


20000.0

In [12]:
#Read many (CRUDE)
projektai = session.query(Projektas).all()
projektai

[(1, Pirmas projektas, 2022-10-15 07:31:00.312290),
 (2, Antras projektas, 2022-10-15 07:31:03.345708),
 (3, Pirmas projektas, 2022-10-15 17:47:43.824175),
 (4, Antras projektas, 2022-10-15 17:47:46.633962)]

#Duomenų paieška pagal salygą

In [13]:
search = session.query(Projektas).filter(Projektas.name.ilike("%as")).all()
# ilike() - key not sencetive search, like() - key sencetive search
search

[(1, Pirmas projektas, 2022-10-15 07:31:00.312290),
 (2, Antras projektas, 2022-10-15 07:31:03.345708),
 (3, Pirmas projektas, 2022-10-15 17:47:43.824175),
 (4, Antras projektas, 2022-10-15 17:47:46.633962)]

In [14]:
search2 = session.query(Projektas).filter(Projektas.price > 25000).all()
search2

[(2, Antras projektas, 2022-10-15 07:31:03.345708),
 (4, Antras projektas, 2022-10-15 17:47:46.633962)]

# Keli parametraiu filtre sujungiami AND sąlyga

In [16]:
search3 = session.query(Projektas).filter(
    Projektas.price > 20000,
    Projektas.name.ilike("Antras%"),
).all()
search3

[(2, Antras projektas, 2022-10-15 07:31:03.345708),
 (4, Antras projektas, 2022-10-15 17:47:46.633962)]

# Update (crud)

In [18]:
atnaujinamas_projektas = session.query(Projektas).get(2)
atnaujinamas_projektas.price = 51500
session.commit()
search2

[(2, Antras projektas, 2022-10-15 07:31:03.345708),
 (4, Antras projektas, 2022-10-15 17:47:46.633962)]

In [23]:
# DELETE
trinamas = session.query(Projektas).filter(Projektas.id == 2).one()
trinamas
session.commit()